<a href="https://colab.research.google.com/github/lil-anchutka/graphcross_grpo/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

#%cd /content/drive/MyDrive/task_graphcross

Mounted at /content/drive


In [ ]:
!pip install nbstripout
nbstripout experiment.ipynb

In [4]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd()
sys.path.insert(0, str(PROJECT_ROOT))

In [5]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd()
sys.path.insert(0, str(PROJECT_ROOT / "src"))

# **Постановка задачи**

<center><img src = 'https://i.imgur.com/OHAF88a.jpeg'  width="500"></center>

Задача представляет собой **синтетическую логическую задачу на восстановление согласованной структуры**, **вдохновлённую кроссвордами**, но не являющуюся кроссвордом в геометрическом смысле и не использующую семантику языка, словари или внешние знания.

Формально задача сводится к работе с **графом взаимосвязей между строками**.
Любой классический кроссворд может быть представлен в таком виде (как набор строк и их пересечений), однако обратное неверно: получающаяся структура **не обязана быть реализуемой как геометрическая сетка**.

Все “слова” в задаче — это **бессмысленные строки из случайных символов**.
Задача не требует знания значений слов или языковых закономерностей и сводится исключительно к выполнению **формальных ограничений согласованности** между элементами структуры.


## **Описание задачи**

* Задан набор **слотов** — абстрактных строк фиксированной длины.
* Каждый слот описывается следующими параметрами:

  * длина строки;
  * множество допустимых кандидатных строк;
* Между слотами задан набор **пересечений**.
  Каждое пересечение задаётся явно как пара:

  * слот A, позиция *i*;
  * слот B, позиция *j*;
    и означает требование равенства символов:
    `A[i] == B[j]`.
* Пересечения образуют **граф связей между слотами**; структура графа произвольна и не предполагает наличия координатной сетки.
* Для каждого слота модели предоставляется **множество кандидатных строк**, из которых необходимо выбрать ровно одну строку для данного слота.


## **Цель**

Требуется выбрать по одной строке для каждого слота так, чтобы одновременно выполнялись следующие условия:

1. Длина выбранной строки совпадает с длиной соответствующего слота.
2. Выбранная строка принадлежит заданному для этого слота множеству кандидатов.
3. Для любого заданного пересечения двух слотов символы выбранных строк в соответствующих позициях **совпадают**.
4. Все выбранные строки совместимы между собой и образуют **глобально согласованное решение**.


## **Сложность задачи**

Сложность задачи регулируется следующими параметрами:

* количеством слотов;
* плотностью пересечений между слотами (структурой графа);
* количеством кандидатных строк на слот;
* числом строк-дистракторов каждого типа (см. ниже).

### **Дистракторы**

> * строки правильной длины, но конфликтующие с другими слотами в одной или нескольких позициях пересечения **(тип 0)**
> * строки, которые согласуются с частью пересечений, но нарушают другие **(тип 1)**
> * строки, которые локально допустимы для конкретного слота, но делают невозможным глобальное согласование всей структуры **(тип 2)**
>
> Это исключает возможность решения задачи с помощью жадного (greedy) выбора локально подходящих вариантов и требует **проверки глобальной совместимости** всех выбранных строк.

Увеличение этих параметров монотонно расширяет пространство поиска и повышает сложность логического вывода, не нарушая гарантированной разрешимости задачи.


## **Соответствие требованиям**

* **Ответ верифицируем.**
  Решение представляет собой явное сопоставление строк слотам. Корректность ответа проверяется формально: каждая выбранная строка должна принадлежать соответствующему множеству кандидатов, иметь корректную длину, а все заданные пересечения — удовлетворяться. Проверка полностью автоматизируема.

* **Сложность задачи регулируема.**
  Сложность контролируется количественными параметрами генерации: числом слотов, плотностью их связей и количеством локально допустимых, но глобально несовместимых кандидатных строк. Увеличение этих параметров расширяет пространство поиска без изменения формулировки правил задачи.

* **Задача решается за один запрос без взаимодействия со средой.**
  Вся информация о слотах, их кандидатах и пересечениях предоставляется модели в одном промпте. Решение не требует пошагового взаимодействия со средой и может быть получено в одном вызове LLM.

---


**Соответствие значений гиперпараметров уровням сложности**


| Difficulty | #Slots | Avg. Intersections / Slot | Candidates / Slot | Distractors (d0 / d1) per slot / d2|
|-----------:|-------:|---------------------------:|-------------------:|----------------------------|
| 1 | 5  | 1.0 – 1.4 | 3 – 4  | 1 / 0 / 0 |
| 2 | 6  | 1.2 – 1.6 | 4 – 5  | 2 / 0 / 0 |
| 3 | 7  | 1.4 – 1.8 | 5 – 6  | 2 / 1 / 0 |
| 4 | 8  | 1.6 – 2.0 | 6 – 7  | 3 / 1 / 0 |
| 5 | 9  | 2.0 – 2.4 | 7 – 8  | 3 / 2 / 1 |
| 6 | 10 | 2.2 – 2.7 | 8 – 10 | 4 / 2 / 1–2 |
| 7 | 12 | 2.6 – 3.2 | 10 – 12 | 4 / 3 / 2 |
| 8 | 13 | 3.0 – 3.6 | 12 – 14 | 5 / 3 / 2–3 |
| 9 | 14 | 3.2 – 3.9 | 14 – 16 | 5 / 4 / 3–4 |
| 10 | 15 | 3.5 – 4.2 | 16 – 20 | 6 / 5 / 4–6 |


значения для дистракторов типа 2 указаны для всего графа, тк. они являются структурными


# Запуск

In [7]:
%%capture
import os
os.environ["UNSLOTH_VLLM_STANDBY"] = "1" # [NEW] Extra 30% context lengths!
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install or uv pip install
    !pip install unsloth vllm
else:
    pass # For Colab / Kaggle, we need extra instructions hidden below \/

In [8]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
!pip install --upgrade -qqq uv
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install!
    !pip install unsloth vllm
else:
    try: import numpy, PIL; get_numpy = f"numpy=={numpy.__version__}"; get_pil = f"pillow=={PIL.__version__}"
    except: get_numpy = "numpy"; get_pil = "pillow"
    try: import subprocess; is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except: is_t4 = False
    get_vllm, get_triton = ("vllm==0.9.2", "triton==3.2.0") if is_t4 else ("vllm==0.10.2", "triton")
    !uv pip install -qqq --upgrade \
        unsloth {get_vllm} {get_numpy} {get_pil} torchvision bitsandbytes xformers
    !uv pip install -qqq {get_triton}
!uv pip install transformers==4.56.2
!uv pip install --no-deps trl==0.22.2

In [5]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
max_seq_length = 3072 # fat ass task :(
lora_rank = 48 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-3B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.8, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

INFO 01-18 13:37:59 [vllm_utils.py:702] Unsloth: Patching vLLM v1 graph capture
INFO 01-18 13:37:59 [vllm_utils.py:731] Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2026.1.3: Fast Qwen2 patching. Transformers: 4.56.2. vLLM: 0.9.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Changing the maximum lora rank to 64 from 48 for vLLM.
Unsloth: vLLM loading unsloth/qwen2.5-3b-instruct-unsloth-bnb-4bit with actual GPU utilization = 79.24%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 3072. Num Sequences = 48.
Unsloth: vLLM's KV Cache can use up to 9.26 GB.

`torch_dtype` is deprecated! Use `dtype` instead!


WARNING 01-18 13:38:32 [config.py:3371] Casting torch.bfloat16 to torch.float16.
INFO 01-18 13:38:32 [config.py:1472] Using max model len 3072
WARNING 01-18 13:38:32 [arg_utils.py:1735] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
INFO 01-18 13:38:35 [config.py:2285] Chunked prefill is enabled with max_num_batched_tokens=4096.
Unsloth: vLLM Bitsandbytes config using kwargs = {'load_in_8bit': False, 'load_in_4bit': True, 'bnb_4bit_compute_dtype': 'float16', 'bnb_4bit_quant_storage': 'uint8', 'bnb_4bit_quant_type': 'nf4', 'bnb_4bit_use_double_quant': True, 'llm_int8_enable_fp32_cpu_offload': False, 'llm_int8_has_fp16_weight': False, 'llm_int8_skip_modules': ['lm_head', 'multi_modal_projector', 'merger', 'modality_projection', 'model.layers.2.mlp', 'model.layers.3.mlp', 'model.layers.30.mlp'], 'llm_int8_threshold': 6.0}
INFO 01-18 13:38:35 [llm_engine.py:230] Initializing a V0 LLM engine (v0.9.2) with config: model='unsloth/qwen2.5-3b-instruct-unsloth-b

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Unsloth 2026.1.3 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [4]:
import os

In [5]:
import json, random, re
from dataclasses import dataclass
from typing import List, Dict, Any, Optional

import torch

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
SRC_DIR = os.path.join(PROJECT_ROOT, "src")
if SRC_DIR not in sys.path:
    sys.path.insert(0, SRC_DIR)

print("PROJECT_ROOT:", PROJECT_ROOT)
print("SRC_DIR:", SRC_DIR)

from graphcross.graphcross import GraphCrossEnv

env = GraphCrossEnv()
print("Env loaded:", env.name)

PROJECT_ROOT: /content/drive/MyDrive
SRC_DIR: /content/drive/MyDrive/src
Env loaded: graphcross


## Генерация и загрузка датасетов

In [ ]:
# do it once
# settings
random.seed(42)

out_root = Path("data/eval")
out_root.mkdir(parents=True, exist_ok=True)

eval_plan = {
    d: 100 if d <= 3 else
       150 if d <= 6 else
       200
    for d in range(1, 11)
}

max_attempts = 800

# generating eval samples

for difficulty, n_samples in eval_plan.items():
    print(f"\n=== Generating eval: difficulty={difficulty}, n={n_samples}")

    data = env.generate(
        num_of_questions=n_samples,
        difficulty=difficulty,
        max_attempts=max_attempts,
    )

    out_path = out_root / f"difficulty_{difficulty}.jsonl"

    with open(out_path, "w", encoding="utf-8") as f:
        for d in data:
            f.write(json.dumps(d.to_json(), ensure_ascii=False) + "\n")

    print(f"[OK] saved to {out_path} ({len(data)} samples)")


=== Generating eval: difficulty=1, n=100
[OK] saved to data/eval/difficulty_1.jsonl (100 samples)

=== Generating eval: difficulty=2, n=100
[OK] saved to data/eval/difficulty_2.jsonl (100 samples)

=== Generating eval: difficulty=3, n=100
[OK] saved to data/eval/difficulty_3.jsonl (100 samples)

=== Generating eval: difficulty=4, n=150
[OK] saved to data/eval/difficulty_4.jsonl (150 samples)

=== Generating eval: difficulty=5, n=150
[OK] saved to data/eval/difficulty_5.jsonl (150 samples)

=== Generating eval: difficulty=6, n=150
[OK] saved to data/eval/difficulty_6.jsonl (150 samples)

=== Generating eval: difficulty=7, n=200
[OK] saved to data/eval/difficulty_7.jsonl (200 samples)

=== Generating eval: difficulty=8, n=200
[OK] saved to data/eval/difficulty_8.jsonl (200 samples)

=== Generating eval: difficulty=9, n=200
[OK] saved to data/eval/difficulty_9.jsonl (200 samples)

=== Generating eval: difficulty=10, n=200
[OK] saved to data/eval/difficulty_10.jsonl (200 samples)


In [6]:
from torch.utils.data import ConcatDataset
from graphcross.datasets import GraphCrossEvalDataset

eval_concat = ConcatDataset([
    GraphCrossEvalDataset(f"data/eval/difficulty_{d}.jsonl")
    for d in range(1, 11)
])

In [ ]:
from graphcross.datasets import GraphCrossTrainDataset

train_dataset = GraphCrossTrainDataset(
    env=env,
    difficulties=list(range(1, 11)),
    n_samples=10000,
    seed=42,
)

# Обучение

### Reward функция/обертка

In [8]:
import re

_FORMAT_SEEN = 0
_FORMAT_FAILS = 0
_WARNED_10 = False
_WARNED_50 = False

# маленькие бонусы (как в референсных ноутбуках)
FORMAT_BONUS = 0.05
THINK_BONUS  = 0.02

_THINK_RE = re.compile(r"<think>\s*[\s\S]*?\s*</think>", re.IGNORECASE)

def correctness_reward_func(prompts, completions, answer, metadata, **kwargs):
    """
    Reward:
      +2.0 if env.verify() == True else +0.0
      +FORMAT_BONUS if output contains parseable JSON (even if semantically wrong)
      +THINK_BONUS if output contains <think>...</think>
    Formatting ValueError -> no crash, counts as format fail, warning on thresholds.
    """
    global _FORMAT_SEEN, _FORMAT_FAILS, _WARNED_10, _WARNED_50

    responses = []
    for c in completions:
        try:
            responses.append(c[0]["content"])
        except Exception:
            responses.append("")

    rewards = []
    for resp, gold, meta in zip(responses, answer, metadata):
        _FORMAT_SEEN += 1

        # build minimal Data-like object expected by verifier
        data = type("Dummy", (), {})()
        data.answer = gold
        data.metadata = meta

        # --- think bonus (независимо от JSON) ---
        think_ok = bool(_THINK_RE.search(resp))
        r = THINK_BONUS if think_ok else 0.0

        # --- JSON parse bonus + correctness ---
        format_ok = False
        ok = False

        try:
            # 1) проверяем, парсится ли JSON (формат)
            #    это даёт бонус даже если семантика не совпала
            _ = env.verifier.extract_answer(resp)  # may raise ValueError
            format_ok = True
            r += FORMAT_BONUS

            # 2) семантическая проверка (может снова парсить, но это ок)
            ok = bool(env.verify(data, resp))

        except ValueError:
            _FORMAT_FAILS += 1

            # warnings at 10 and 50 (for ~10k dataset: 0.1% and 0.5%)
            if (not _WARNED_10) and _FORMAT_FAILS >= 10:
                _WARNED_10 = True
                rate = _FORMAT_FAILS / max(1, _FORMAT_SEEN)
                print(f"[WARN] JSON format failures >= 10 (fails={_FORMAT_FAILS}/{_FORMAT_SEEN} = {rate:.2%})")

            if (not _WARNED_50) and _FORMAT_FAILS >= 50:
                _WARNED_50 = True
                rate = _FORMAT_FAILS / max(1, _FORMAT_SEEN)
                print(f"[WARN] JSON format failures >= 50 (fails={_FORMAT_FAILS}/{_FORMAT_SEEN} = {rate:.2%})")

            # ok остаётся False, format_ok False

        # основной reward за правильность
        if ok:
            r += 2.0

        rewards.append(float(r))

    return rewards


In [ ]:
from trl import GRPOTrainer, GRPOConfig

# training args (same as the reference notebook)
training_args = GRPOConfig(
    output_dir="outputs_graphcross_grpo",
    logging_steps=10,
    save_steps=200,
    save_total_limit=2,

    learning_rate=5e-6,
    warmup_steps=50,
    max_steps=500,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,

    max_prompt_length=1024,
    max_completion_length=512,
    num_generations=2,

    bf16=torch.cuda.is_available() and torch.cuda.get_device_capability()[0] >= 8,
    fp16=not (torch.cuda.is_available() and torch.cuda.get_device_capability()[0] >= 8),
    report_to="none",
)

In [ ]:
# Trainer
trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[
        correctness_reward_func
    ],
    args=training_args,
    train_dataset=train_dataset,
)

trainer.train()

trainer.model.save_pretrained("graphcross_grpo_lora")
tokenizer.save_pretrained("graphcross_grpo_lora")

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10,000 | Num Epochs = 1 | Total steps = 500
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 2 x 1) = 4
 "-____-"     Trainable parameters = 89,800,704 of 3,175,739,392 (2.83% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / correctness_reward_func / mean,rewards / correctness_reward_func / std
10,0.000000,0.060000,0.005657,333.700000,232.200000,426.200000,0.175000,299.358334,232.200000,367.300000,0.000024,0.060000,0.014600
20,0.000000,0.064000,0.008485,292.825000,188.300000,400.100000,0.125000,263.441669,188.300000,335.500000,0.000023,0.064000,0.012000
30,0.000000,0.057250,0.009546,296.500000,189.600000,450.500000,0.175000,245.558336,189.600000,332.500000,0.000062,0.057250,0.017455
40,0.000000,0.065000,0.007071,317.125000,189.100000,446.700000,0.050000,307.908337,189.100000,425.800000,0.000355,0.065000,0.010000
50,0.000000,0.065250,0.006718,306.025000,215.000000,438.600000,0.100000,277.925000,215.000000,372.700000,0.004272,0.065250,0.006541
60,0.000000,0.061750,0.003182,294.575000,194.400000,380.100000,0.125000,262.516669,194.400000,331.800000,0.005816,0.061750,0.011100
70,0.000000,0.060750,0.013081,308.600000,194.800000,462.000000,0.125000,274.741669,194.800000,362.000000,0.009634,0.060750,0.018500
80,0.000000,0.063000,0.009899,268.150000,171.300000,448.100000,0.125000,229.950002,171.300000,319.000000,0.019202,0.063000,0.013155
90,0.000000,0.066250,0.005303,233.300000,151.600000,340.500000,0.075000,213.475002,151.600000,288.400000,0.027881,0.066250,0.007500
100,0.000000,0.068250,0.002475,210.275000,173.000000,266.700000,0.025000,201.808334,173.000000,234.200000,0.046348,0.068250,0.003500


[WARN] JSON format failures >= 10 (fails=10/79 = 12.66%)


KeyboardInterrupt: 

Из логов выше (хоть и незавершенных, у меня палец соскользнул) видим, что обучение никуда не двигается. Возможно в текущей конфигурации "нащупать решение" для модели слишком сложно - даже случайный шанс попасть в верную расстановку очень мал (при сложности 1 например, это около 0.0039)

Попробуем пересмотреть подход. Упростим набор задач на которых будет обучаться модель и разобьем тренинг в два этапа. Ниже новая таблица соответствия сложностей и гиперпараметров задачи

| Difficulty | #Slots | Avg. Intersections / Slot | Candidates / Slot | Distractors (d0 / d1 / d2) | Назначение уровня                         |
| ---------: | -----: | ------------------------: | ----------------: | -------------------------- | ----------------------------------------- |
|      **0** |    3–4 |                   1.5–2.0 |                 2 | 0 / 0 / 0                  | Warm-up: формат + базовая согласованность |
|      **1** |    4–5 |                   1.8–2.2 |               2–3 | 0 / 0 / 0                  | Чистая constraint propagation             |
|      **2** |    5–6 |                   2.0–2.5 |                 3 | 1 / 0 / 0                  | Первый локальный конфликт                 |
|      **3** |    6–7 |                   2.2–2.8 |               3–4 | 1 / 1 / 0                  | Частичная глобальность                    |
|      **4** |    7–8 |                   2.5–3.0 |                 4 | 2 / 1 / 0                  | Конфликты без поиска                      |
|      **5** |    8–9 |                   2.8–3.3 |               4–5 | 2 / 2 / 1                  | Первый обязательный backtracking          |
|      **6** |   9–10 |                   3.0–3.6 |               5–6 | 3 / 2 / 1–2                | Глобальные ловушки                        |
|      **7** |  11–12 |                   3.3–3.9 |               6–7 | 3 / 3 / 2                  | Hard CSP                                  |
|      **8** |  12–13 |                   3.6–4.2 |               7–8 | 4 / 3 / 2–3                | Very hard                                 |
|      **9** |     14 |                   4.0–4.6 |              9–10 | 4 / 4 / 3–4                | Extreme                                   |
|     **10** |     15 |                   4.5–5.2 |             10–12 | 5 / 5 / 4–6                | Stress test                               |


In [ ]:
from trl import GRPOTrainer, GRPOConfig
from graphcross.datasets import GraphCrossTrainDataset

# dataset for the 1st stage of training
# proportions: 50 / 30 / 20
N_STAGE1 = 6000

ds0 = GraphCrossTrainDataset(
    env=env,
    difficulties=[0],
    n_samples=int(N_STAGE1 * 0.50),
    seed=42,
)

ds1 = GraphCrossTrainDataset(
    env=env,
    difficulties=[1],
    n_samples=int(N_STAGE1 * 0.30),
    seed=43,
)

ds2 = GraphCrossTrainDataset(
    env=env,
    difficulties=[2],
    n_samples=int(N_STAGE1 * 0.20),
    seed=44,
)

train_dataset_stage1 = ConcatDataset([ds0, ds1, ds2])

In [ ]:
training_args_stage1 = GRPOConfig(
    output_dir="outputs_graphcross_stage1",
    logging_steps=10,
    save_steps=200,
    save_total_limit=2,

    learning_rate=5e-6,
    warmup_steps=50,
    max_steps=600,

    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,

    max_prompt_length=1024,
    max_completion_length=512,
    num_generations=2,

    bf16=torch.cuda.is_available() and torch.cuda.get_device_capability()[0] >= 8,
    fp16=not (torch.cuda.is_available() and torch.cuda.get_device_capability()[0] >= 8),
    report_to="none",
)

In [ ]:
trainer1 = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[correctness_reward_func],
    args=training_args_stage1,
    train_dataset=train_dataset_stage1,
)

trainer1.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 6,000 | Num Epochs = 1 | Total steps = 600
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 89,800,704 of 3,175,739,392 (2.83% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / correctness_reward_func / mean,rewards / correctness_reward_func / std
10,0.000000,0.054000,0.022627,358.775000,197.400000,491.000000,0.275000,298.416672,197.400000,415.200000,0.000017,0.054000,0.024487
20,0.000000,0.038500,0.020506,414.200000,279.900000,505.800000,0.475000,297.500003,228.700000,357.200000,0.000023,0.038500,0.027759
30,0.000000,0.046500,0.016263,359.275000,192.700000,477.200000,0.350000,275.700003,192.700000,357.700000,0.000280,0.046500,0.024481
40,0.000000,0.053750,0.018031,326.050000,161.800000,490.400000,0.250000,267.441670,161.800000,389.900000,0.001160,0.053750,0.024709
50,0.000000,0.063500,0.009192,217.075000,113.700000,363.900000,0.100000,185.041669,113.700000,278.500000,0.006146,0.063500,0.013000
60,0.000000,0.070000,0.000000,161.675000,113.200000,253.500000,0.000000,161.675000,113.200000,253.500000,0.009525,0.070000,0.000000
70,0.000000,0.066500,0.004950,213.525000,119.300000,361.100000,0.075000,186.966667,119.300000,276.600000,0.015037,0.066500,0.007000
80,0.000000,0.068250,0.002475,170.550000,114.000000,272.500000,0.050000,152.625000,114.000000,212.600000,0.011285,0.068250,0.003500
90,0.000000,0.069500,0.000707,146.375000,110.200000,199.200000,0.000000,146.375000,110.200000,199.200000,0.012354,0.069500,0.001000
100,0.000000,0.070000,0.000000,140.600000,112.500000,187.200000,0.000000,140.600000,112.500000,187.200000,0.019735,0.070000,0.000000


[WARN] JSON format failures >= 10 (fails=10/39 = 25.64%)
[WARN] JSON format failures >= 50 (fails=50/150 = 33.33%)


TrainOutput(global_step=600, training_loss=1.742804353848252e-05, metrics={'train_runtime': 12410.433, 'train_samples_per_second': 0.193, 'train_steps_per_second': 0.048, 'total_flos': 0.0, 'train_loss': 1.742804353848252e-05})

In [ ]:
trainer1.model.save_pretrained("graphcross_grpo_stage1_lora")
tokenizer.save_pretrained("graphcross_grpo_stage1_lora")

('graphcross_grpo_stage1_lora/tokenizer_config.json',
 'graphcross_grpo_stage1_lora/special_tokens_map.json',
 'graphcross_grpo_stage1_lora/chat_template.jinja',
 'graphcross_grpo_stage1_lora/vocab.json',
 'graphcross_grpo_stage1_lora/merges.txt',
 'graphcross_grpo_stage1_lora/added_tokens.json',
 'graphcross_grpo_stage1_lora/tokenizer.json')

# Оценка модели

Ранее мы наблюдали, что даже при максимально упрощенной конфигурации задач модель не получает обучающего сигнала на train выборке.
Из этих соображений раздел evaluation опускается за ненадобностью.

<img src="https://i.imgur.com/qxn624h.jpeg">

# Выводы

<center><img src="https://preview.redd.it/pi38ojxhy3801.png?auto=webp&s=9da1f554e316d41251505a76412e5b2d92aec33d"  width="300"/> <p><em>Figure X: Illustration of reinforcement learning dynamics under sparse binary reward.</em></p></center>



Возможной причиной того, что модель не смогла обучиться даже на максимально упрощённой версии обучающей выборки, является сочетание разреженного бинарного вознаграждения и крайне негладкой, условно «игольчатой», поверхности ожидаемой награды. В таких условиях вероятность случайно получить корректное решение остаётся очень низкой, из-за чего модель практически не получает обучающего сигнала.

Для примера, в задачах GSM8K, использованных в референсном ноутбуке, аналогичная модель демонстрирует обучение при бинарном reward, что, вероятно, связано с наличием сильных априорных знаний. Модель уже на этапе претрейна усваивает семантические и арифметические паттерны, которые структурируют пространство решений и позволяют постепенно приближаться к правильному ответу. Кроме того, сами текстовые условия таких задач содержат семантическую информацию, которая делает пространство допустимых продолжений более информативным.

Задача Graph Cross, напротив, намеренно лишена языковой и семантической структуры. Кандидатные строки являются случайными, а ограничения задаются исключительно формальными индексными равенствами. В результате корректные решения оказываются изолированными точками в пространстве возможных ответов и не образуют плавного перехода от частично корректных конфигураций к полностью согласованному решению. В таких условиях методы policy gradient не получают информативного направления для обновления политики.


